In [2]:
from utils import Dataset
from utils.helpers import getDatasetPath
from constants import TYPES, KINGDOMS
import pickle

In [4]:
all_data = Dataset(getDatasetPath()).getFolds([0, 1, 2, 3, 4])

In [28]:
counts = all_data.groupby(["type", "kingdom"]).agg("count")
counts["count"] = counts["sequence"]
counts["fraction"] = counts["sequence"].map(lambda x: x/n)
counts = counts[["count", "fraction"]]
counts.sort_values(by="count")

,,count,fraction
type,kingdom,,
TAT,ARCHAEA,27,0.001313
LIPO,ARCHAEA,28,0.001361
SP,ARCHAEA,60,0.002917
TAT,POSITIVE,95,0.004618
NO_SP,ARCHAEA,119,0.005785
SP,POSITIVE,184,0.008945
NO_SP,POSITIVE,188,0.009139
TAT,NEGATIVE,334,0.016236
NO_SP,NEGATIVE,422,0.020514


In [6]:
all_annotations = "".join([ann for ann in all_data["annotation"]])
test_annotations = "".join([ann for ann in all_data.query("partition == 0")["annotation"]])

In [21]:
n = all_data.shape[0]

(20571, 2)

In [10]:
type_proportions = {
    type: all_data.query(f"type == '{type}'").shape[0] / n
    for type in TYPES
}

kingdom_proportions = {
    kingdom: all_data.query(f"kingdom == '{kingdom}'").shape[0] / n
    for kingdom in KINGDOMS
}

label_proportions = {
    label: test_annotations.count(label) / len(test_annotations)
    for label in [c for c in "STLIMO"]
}

In [11]:
for filename, dict in zip(["pathway_distribution", "kingdom_distribution", "label_distribution"], [type_proportions, kingdom_proportions, label_proportions]):
    with open(f"../{filename}.pickle", "wb+") as f:
        pickle.dump(dict, f)

In [12]:
kingdom_by_type = {
    type: {
        kingdom: all_data.query(f"type == '{type}' and kingdom == '{kingdom}'").shape[0] / all_data.query(f"type == '{type}'").shape[0]
        for kingdom in KINGDOMS
    } for type in TYPES
}
kingdom_by_type

{'SP': {'EUKARYA': 0.7696078431372549,
  'ARCHAEA': 0.01838235294117647,
  'POSITIVE': 0.056372549019607844,
  'NEGATIVE': 0.1556372549019608},
 'NO_SP': {'EUKARYA': 0.9524833789597185,
  'ARCHAEA': 0.007756485464737322,
  'POSITIVE': 0.012253943423282492,
  'NEGATIVE': 0.027506192152261764},
 'TAT': {'EUKARYA': 0.0,
  'ARCHAEA': 0.05921052631578947,
  'POSITIVE': 0.20833333333333334,
  'NEGATIVE': 0.7324561403508771},
 'LIPO': {'EUKARYA': 0.0,
  'ARCHAEA': 0.01855533465871438,
  'POSITIVE': 0.2962226640159046,
  'NEGATIVE': 0.685222001325381}}

In [13]:
type_by_kingdom = {
    kingdom: {
        type : all_data.query(f"type == '{type}' and kingdom == '{kingdom}'").shape[0] / all_data.query(f"kingdom == '{kingdom}'").shape[0]
        for type in TYPES
    }  for kingdom in KINGDOMS
}

type_by_kingdom

{'EUKARYA': {'SP': 0.14668613138686132,
  'NO_SP': 0.8533138686131387,
  'TAT': 0.0,
  'LIPO': 0.0},
 'ARCHAEA': {'SP': 0.2564102564102564,
  'NO_SP': 0.5085470085470085,
  'TAT': 0.11538461538461539,
  'LIPO': 0.11965811965811966},
 'POSITIVE': {'SP': 0.2013129102844639,
  'NO_SP': 0.20568927789934355,
  'TAT': 0.10393873085339168,
  'LIPO': 0.4890590809628009},
 'NEGATIVE': {'SP': 0.22106179286335945,
  'NO_SP': 0.1836379460400348,
  'TAT': 0.1453437771975631,
  'LIPO': 0.44995648389904264}}